---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Ridge


def blight_model():
    df_train=pd.read_csv("train.csv", encoding='cp1252')
    df_test=pd.read_csv("readonly/test.csv").set_index('ticket_id')
    df_train=df_train.dropna(subset=['compliance'], inplace=False)
    
    df_add=pd.read_csv("readonly/addresses.csv")
    df_lat=pd.read_csv("readonly/latlons.csv")

    df_add=df_add.set_index('address').join(df_lat.set_index('address'), how='left')
    df_add=df_add.reset_index()
    df_add=df_add.drop_duplicates(subset=['address','lat','lon'])
    
    df_train=df_train.merge(df_add, how='left')
    df_train=df_train.dropna(subset=['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'address', 'lat','lon'], inplace=False)
    df_train=df_train.drop(['agency_name', 'inspector_name', 'violator_name','violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city'
              , 'state', 'non_us_str_code', 'country', 'violation_description', 'payment_status', 'grafitti_status', 'compliance_detail'], axis=1)
    
    df_train=df_train.drop(['balance_due', 'collection_status', 'payment_amount', 'payment_date', 'judgment_amount', 'violation_street_number', 'zip_code', 'ticket_issued_date', 'hearing_date', 'disposition', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost'], axis=1)
    df_test=df_test.dropna(subset=['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount'], inplace=False)
    df_test=df_test.drop(['violator_name', 'zip_code', 'country', 'city',
            'inspector_name', 'violation_street_number', 'violation_street_name',
            'violation_zip_code', 'violation_description',
            'mailing_address_str_number', 'mailing_address_str_name',
            'non_us_str_code', 'agency_name', 'state', 'disposition',
            'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code'], axis=1)
    df_test=df_test.drop(['admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount'], axis=1)
    df_test=df_test.reset_index()
    df_test=df_test.merge(df_add, how='left')
    df_test=df_test.set_index('ticket_id')
    df_train=df_train.set_index('ticket_id')
    
    y_train=df_train['compliance']
    X_train=df_train.drop(['compliance','violation_code','address'], axis=1)
    
    X_test=df_test.drop(['address'], axis=1)
    
    X_test.lat.fillna(method='pad', inplace=True)
    X_test.lon.fillna(method='pad', inplace=True)
    X_train.lat.fillna(method='pad', inplace=True)
    X_train.lon.fillna(method='pad', inplace=True)
    
    scaler=MinMaxScaler()
    
    X_train_scaled=scaler.fit_transform(X_train)
    X_test_scaled=scaler.fit_transform(X_test)
    
    lin_reg=LogisticRegression().fit(X_train_scaled, y_train)
    
    y_proba=lin_reg.predict_proba(X_test_scaled)
    
    final_df=pd.DataFrame(data=y_proba)
    df_test=df_test.reset_index()
    
    final_df['ticket_id']=df_test['ticket_id']
    final_df=final_df.set_index('ticket_id')
    
    final_df['compliance']=final_df[0]
    final_df=final_df.drop([0,1], axis=1)
    
    return final_df.compliance# Your answer here

In [83]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.983348
285362    0.991752
285361    0.986944
285338    0.946373
285346    0.943695
285345    0.946373
285347    0.942309
285342    0.946373
285530    0.963844
284989    0.969182
285344    0.945083
285343    0.965621
285340    0.965621
285341    0.945083
285349    0.942825
285348    0.945542
284991    0.952980
285532    0.952980
285406    0.952980
285001    0.952980
285006    0.963274
285405    0.963274
285337    0.952980
285496    0.941420
285497    0.945542
285378    0.992435
285589    0.990232
285585    0.988617
285501    0.988022
285581    0.992435
            ...   
376367    0.986657
376366    0.982800
376362    0.982800
376363    0.982358
376365    0.986657
376364    0.982800
376228    0.965410
376265    0.983186
376286    0.987562
376320    0.972005
376314    0.974660
376327    0.974660
376385    0.974660
376435    0.969447
376370    0.960813
376434    0.986880
376459    0.969087
376478    0.997457
376473    0.971321
376484    0.972031
376482    0.989589
37

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df_train=pd.read_csv("train.csv", encoding='cp1252')
df_test=pd.read_csv("readonly/test.csv").set_index('ticket_id')
df_train=df_train.dropna(subset=['compliance'], inplace=False)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_add=pd.read_csv("readonly/addresses.csv")
df_lat=pd.read_csv("readonly/latlons.csv")

df_add=df_add.set_index('address').join(df_lat.set_index('address'), how='left')
df_add=df_add.reset_index()
df_add=df_add.drop_duplicates(subset=['address','lat','lon'])

In [3]:
df_train=df_train.merge(df_add, how='left')

In [4]:
df_train=df_train.dropna(subset=['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'address', 'lat','lon'], inplace=False)

In [5]:
df_train=df_train.drop(['agency_name', 'inspector_name', 'violator_name','violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city'
              , 'state', 'non_us_str_code', 'country', 'violation_description', 'payment_status', 'grafitti_status', 'compliance_detail'], axis=1)

In [6]:
df_train=df_train.drop(['balance_due', 'collection_status', 'payment_amount', 'payment_date', 'judgment_amount', 'violation_street_number', 'zip_code', 'ticket_issued_date', 'hearing_date', 'disposition', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost'], axis=1)

In [7]:
df_test=df_test.dropna(subset=['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount'], inplace=False)
df_test=df_test.drop(['violator_name', 'zip_code', 'country', 'city',
        'inspector_name', 'violation_street_number', 'violation_street_name',
        'violation_zip_code', 'violation_description',
        'mailing_address_str_number', 'mailing_address_str_name',
        'non_us_str_code', 'agency_name', 'state', 'disposition',
        'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code'], axis=1)

In [8]:
df_test=df_test.drop(['admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount'], axis=1)

In [9]:
df_test=df_test.reset_index()

In [10]:
df_test=df_test.merge(df_add, how='left')

In [11]:
df_test=df_test.set_index('ticket_id')
df_train=df_train.set_index('ticket_id')

In [12]:
y_train=df_train['compliance']
X_train=df_train.drop(['compliance','violation_code','address'], axis=1)

In [13]:
X_test=df_test.drop(['address'], axis=1)


In [14]:
X_test.lat.fillna(method='pad', inplace=True)

In [15]:
X_test.lon.fillna(method='pad', inplace=True)

In [16]:
X_train.lat.fillna(method='pad', inplace=True)

In [17]:
X_train.lon.fillna(method='pad', inplace=True)

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
scaler=MinMaxScaler()

In [20]:
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Ridge

logreg=LogisticRegression()
rand=RandomForestClassifier(n_estimators=5, random_state=0)
grad=GradientBoostingClassifier(random_state=0)
rand_reg=RandomForestRegressor()

score_logreg=cross_val_score(logreg, X_train, y_train, cv=5)
score_rand=cross_val_score(rand, X_train, y_train, cv=5)
score_grad=cross_val_score(grad, X_train, y_train, cv=5)
score_rand_reg=cross_val_score(rand_reg, X_train, y_train, cv=5)

print("Cross Validation scores for Logistic Regression {}".format(score_logreg.mean()))
print("Cross Validation scores for Random Class {}".format(score_rand.mean()))
print("Cross Validation scores for GradBoost {}".format(score_grad.mean()))
print("Cross Validation scores for Random Regressor {}".format(score_rand_reg.mean()))


Cross Validation scores for Logistic Regression 0.9007559916358371
Cross Validation scores for Random Class 0.8691651922148946
Cross Validation scores for GradBoost 0.9006433971368827
Cross Validation scores for Random Regressor -0.2260455443289588


In [69]:
lin_reg=LogisticRegression().fit(X_train_scaled, y_train)
y_proba=lin_reg.predict_proba(X_test_scaled)

In [76]:
final_df=pd.DataFrame(data=y_proba)
final_df['ticket_id']=df_test['ticket_id']

In [77]:
final_df=final_df.set_index('ticket_id')

In [78]:
final_df.head()

,0,1
ticket_id,,
284932,0.983348,0.016652
285362,0.991752,0.008248
285361,0.986944,0.013056
285338,0.946373,0.053627
285346,0.943695,0.056305


In [81]:
final_df['compliance']=final_df[0]
final_df=final_df.drop([0,1], axis=1)
final_df

,compliance
ticket_id,
284932,0.983348
285362,0.991752
285361,0.986944
285338,0.946373
285346,0.943695
285345,0.946373
285347,0.942309
285342,0.946373
285530,0.963844
